<center> <img src = https://raw.githubusercontent.com/AndreyRysistov/DatasetsForPandas/main/hh%20label.jpg alt="drawing" style="width:400px;">

# <center> Проект: Анализ вакансий из HeadHunter
   

In [1]:
import pandas as pd
from sqlalchemy import create_engine
from dotenv import load_dotenv
import os

In [2]:
load_dotenv()

DBNAME = os.getenv('DBNAME')
USER = os.getenv('DBUSER')
PASSWORD = os.getenv('PASSWORD')
HOST = os.getenv('HOST')
PORT = os.getenv('PORT')

In [3]:
engine = create_engine(f"postgresql+psycopg2://{USER}:{PASSWORD}@{HOST}:{PORT}/{DBNAME}")

# Юнит 3. Предварительный анализ данных

1. Напишите запрос, который посчитает количество вакансий в нашей базе (вакансии находятся в таблице vacancies). 

In [4]:
# текст запроса
query_3_1 = f'''
select count(*)
from vacancies;
'''

In [5]:
# результат запроса
df = pd.read_sql_query(query_3_1, engine)
print('Количество вакансий:', df['count'].values[0])

Количество вакансий: 49197


2. Напишите запрос, который посчитает количество работодателей (таблица employers). 

In [63]:
# текст запроса
query_3_2 = f'''
select count(*)
from employers;
'''

In [64]:
# результат запроса
df = pd.read_sql_query(query_3_2, engine)
print('Количество работодателей:', df['count'].values[0])

Количество работодателей: 23501


3. Посчитате с помощью запроса количество регионов (таблица areas).

In [65]:
# текст запроса
query_3_3 = f'''
select count(*)
from areas;
'''

In [66]:
# результат запроса
df = pd.read_sql_query(query_3_3, engine)
print('Количество регионов:', df['count'].values[0])

Количество регионов: 1362


4. Посчитате с помощью запроса количество сфер деятельности в базе (таблица industries).

In [67]:
# текст запроса
query_3_4 = f'''
select count(*)
from industries;
'''

In [68]:
# результат запроса
df = pd.read_sql_query(query_3_4, engine)
print('Количество сфер деятельности:', df['count'].values[0])

Количество сфер деятельности: 294


### Выводы по предварительному анализу данных
1. В базе данных:
    - **Вакансий** — в 2 раза больше, чем работодателей.
    - Представлено большое количество **регионов**.
    - Отражено разнообразие **сфер деятельности**.

2. Вакансии, вероятно, охватывают всю Россию.
3. База данных готова для дальнейшего детального анализа.


# Юнит 4. Детальный анализ вакансий

1. Напишите запрос, который позволит узнать, сколько (cnt) вакансий в каждом регионе (area).
Отсортируйте по количеству вакансий в порядке убывания.

In [69]:
# текст запроса
query_4_1 = '''
select a.name, count(v.id) as cnt
from vacancies v
         join areas a on v.area_id = a.id
group by a.name
order by cnt desc
limit 5;
'''

In [70]:
# результат запроса
df = pd.read_sql_query(query_4_1, engine)
print('Количество вакансий в каждом регионе:\n', df)

Количество вакансий в каждом регионе:
               name   cnt
0           Москва  5333
1  Санкт-Петербург  2851
2            Минск  2112
3      Новосибирск  2006
4           Алматы  1892


2. Напишите запрос, чтобы определить у какого количества вакансий заполнено хотя бы одно из двух полей с зарплатой.

In [71]:
# текст запроса
query_4_2 = '''
select count(*)
from vacancies
where salary_from is not null
   or salary_to is not null;
'''

In [72]:
# результат запроса
df = pd.read_sql_query(query_4_2, engine)
print('Количество вакансий заполнено хотя бы одно из двух полей с зарплатой:', df['count'].values[0])

Количество вакансий заполнено хотя бы одно из двух полей с зарплатой: 24073


3. Найдите средние значения для нижней и верхней границы зарплатной вилки. Округлите значения до целого.

In [73]:
# текст запроса
query_4_3 = '''
select round(avg(salary_from)) as avg_min,
       round(avg(salary_to))   as avg_max
from vacancies;
'''

In [74]:
# результат запроса
df = pd.read_sql_query(query_4_3, engine)
print('Средние значения для нижней и верхней границы зарплатной вилки:\n', df)

Средние значения для нижней и верхней границы зарплатной вилки:
    avg_min   avg_max
0  71065.0  110537.0


4. Напишите запрос, который выведет количество вакансий для каждого сочетания типа рабочего графика (schedule) и типа трудоустройства (employment), используемого в вакансиях. Результат отсортируйте по убыванию количества.


In [75]:
# текст запроса
query_4_4 = '''
select schedule, employment, count(*) as count
from vacancies
group by schedule, employment
order by count desc;
'''

In [76]:
# результат запроса
df = pd.read_sql_query(query_4_4, engine)
print('Пара находится на втором месте по популярности:\n', df.iloc[1])

Пара находится на втором месте по популярности:
 schedule      Удаленная работа
employment    Полная занятость
count                     7802
Name: 1, dtype: object


5. Напишите запрос, выводящий значения поля Требуемый опыт работы (experience) в порядке возрастания количества вакансий, в которых указан данный вариант опыта. 

In [77]:
# текст запроса
query_4_5 = '''
select experience, count(*) as count
from vacancies
group by experience
order by count;
'''

In [78]:
# результат запроса
df = pd.read_sql_query(query_4_5, engine)
print('Порядок опыта:\n', df)

Порядок опыта:
            experience  count
0         Более 6 лет   1337
1           Нет опыта   7197
2       От 3 до 6 лет  14511
3  От 1 года до 3 лет  26152


***

### Выводы по детальному анализу вакансий
1. **Распределение вакансий**: Большинство вакансий сосредоточено в крупных экономических центрах.
2. **Заполненность данных**: У значительной части вакансий указаны данные о зарплате.
3. **Средние значения зарплат**: Средние границы зарплат варьируются в разумных пределах.
4. **Типы занятости и графики работы**: Полный рабочий день и постоянная занятость — самые популярные форматы.
5. **Требуемый опыт работы**: Вакансии чаще ориентированы на кандидатов с небольшим или отсутствующим опытом.



# Юнит 5. Анализ работодателей

1. Напишите запрос, который позволит узнать, какие работодатели находятся на первом и пятом месте по количеству вакансий.

In [79]:
# текст запроса
query_5_1 = '''
select e.name, count(v.id) as count
from vacancies v
join employers e on e.id = v.employer_id
group by e.name
order by count desc
limit 5;
'''

In [80]:
# результат запроса
df = pd.read_sql_query(query_5_1, engine)
print('Работодатели находятся на первом и пятом месте по количеству вакансий:\n', '1', df.iloc[0]['name'], '\n', '5',
      df.iloc[4]['name'])

Работодатели находятся на первом и пятом месте по количеству вакансий:
 1 Яндекс 
 5 Газпром нефть


2. Напишите запрос, который для каждого региона выведет количество работодателей и вакансий в нём.
Среди регионов, в которых нет вакансий, найдите тот, в котором наибольшее количество работодателей.


In [81]:
# текст запроса
query_5_2 = '''
select a.name,
       count(e.id) as emp_count,
       count(v.id) as vac_count
from areas a
         left join employers e on e.area = a.id
         left join vacancies v on a.id = v.area_id
group by a.name
order by vac_count, emp_count desc
limit 1;
'''

In [82]:
# результат запроса
df = pd.read_sql_query(query_5_2, engine)
print('Регион без вакансий с наибольшим количеством работодателей:', df['name'].values[0])

Регион без вакансий с наибольшим количеством работодателей: Россия


3. Для каждого работодателя посчитайте количество регионов, в которых он публикует свои вакансии. Отсортируйте результат по убыванию количества.


In [83]:
# текст запроса
query_5_3 = '''
select e.name,
       count(distinct v.area_id) as area_count
from employers e
         join areas a on e.area = a.id
         join vacancies v on e.id = v.employer_id
group by e.name
order by area_count desc
limit 1;
'''

In [84]:
# результат запроса
df = pd.read_sql_query(query_5_3, engine)
print('Максимальное значение:', df['area_count'].values[0])

Максимальное значение: 181


4. Напишите запрос для подсчёта количества работодателей, у которых не указана сфера деятельности. 

In [85]:
# текст запроса
query_5_4 = '''
select count(*)
from employers
         left join employers_industries ei on employers.id = ei.employer_id
where ei.industry_id is null;
'''

In [86]:
# результат запроса
df = pd.read_sql_query(query_5_4, engine)
print('Работодатели у которых не указана сфера деятельности:', df['count'].values[0])

Работодатели у которых не указана сфера деятельности: 8419


5. Напишите запрос, чтобы узнать название компании, находящейся на третьем месте в алфавитном списке (по названию) компаний, у которых указано четыре сферы деятельности. 

In [87]:
# текст запроса
query_5_5 = '''
select name
from employers
         join employers_industries ei on employers.id = ei.employer_id
group by name
having count(ei.industry_id) = 4
order by name
offset 2 limit 1;
'''

In [88]:
# результат запроса
df = pd.read_sql_query(query_5_5, engine)
print(
    'Компании, находящейся на третьем месте в алфавитном списке (по названию) компаний, у которой указано четыре сферы деятельности:',
    df['name'].values[0])

Компании, находящейся на третьем месте в алфавитном списке (по названию) компаний, у которой указано четыре сферы деятельности: 2ГИС


6. С помощью запроса выясните, у какого количества работодателей в качестве сферы деятельности указана Разработка программного обеспечения.


In [89]:
# текст запроса
query_5_6 = '''
select count(e.id)
from employers e
         join employers_industries ei on e.id = ei.employer_id
         join industries i on i.id = ei.industry_id
where i.name = 'Разработка программного обеспечения';
'''

In [90]:
# результат запроса
df = pd.read_sql_query(query_5_6, engine)
print('Количества работодателей у которых в качестве сферы деятельности указана «Разработка программного обеспечения»:',
      df['count'].values[0])

Количества работодателей у которых в качестве сферы деятельности указана «Разработка программного обеспечения»: 3553


7. Для компании «Яндекс» выведите список регионов-миллионников, в которых представлены вакансии компании, вместе с количеством вакансий в этих регионах. Также добавьте строку Total с общим количеством вакансий компании. Результат отсортируйте по возрастанию количества.

Список городов-милионников надо взять [отсюда](https://ru.wikipedia.org/wiki/%D0%93%D0%BE%D1%80%D0%BE%D0%B4%D0%B0-%D0%BC%D0%B8%D0%BB%D0%BB%D0%B8%D0%BE%D0%BD%D0%B5%D1%80%D1%8B_%D0%A0%D0%BE%D1%81%D1%81%D0%B8%D0%B8). 

Если возникнут трудности с этим задание посмотрите материалы модуля  PYTHON-17. Как получать данные из веб-источников и API. 

In [91]:
# код для получения списка городов-милионников
import requests
from bs4 import BeautifulSoup

url = "https://ru.wikipedia.org/wiki/%D0%93%D0%BE%D1%80%D0%BE%D0%B4%D0%B0-%D0%BC%D0%B8%D0%BB%D0%BB%D0%B8%D0%BE%D0%BD%D0%B5%D1%80%D1%8B_%D0%A0%D0%BE%D1%81%D1%81%D0%B8%D0%B8"

response = requests.get(url)
response.encoding = "utf-8"
html = response.text

soup = BeautifulSoup(html, "html.parser")

table = soup.find("table", {"class": "standard"})

rows = table.find_all("tr")

cities = []
for row in rows[1:]:
    cols = row.find_all("td")
    if len(cols) > 0:
        city = cols[1].text.strip()
        cities.append(city)

sql_cities = str(tuple(cities))

In [92]:
# текст запроса
query_5_7 = f'''
select a.name      as name,
       count(v.id) as cnt
from vacancies v
         join areas a on a.id = v.area_id
         join employers e on v.employer_id = e.id
where e.name = 'Яндекс'
  and a.name in {sql_cities}
group by a.name
union
select 'Total',
       count(v.id)
from vacancies v
         join areas a on a.id = v.area_id
         join employers e on v.employer_id = e.id
where e.name = 'Яндекс'
  and a.name in {sql_cities}
order by cnt
'''

In [93]:
# результат запроса
df = pd.read_sql_query(query_5_7, engine)
print('Количества вакансий по регионам для компании Яндекс:\n', df)

Количества вакансий по регионам для компании Яндекс:
                name  cnt
0              Омск   21
1         Челябинск   22
2        Красноярск   23
3         Волгоград   24
4             Пермь   25
5            Казань   25
6    Ростов-на-Дону   25
7               Уфа   26
8            Самара   26
9         Краснодар   30
10          Воронеж   32
11      Новосибирск   35
12  Нижний Новгород   36
13     Екатеринбург   39
14  Санкт-Петербург   42
15           Москва   54
16            Total  485


***

### Выводы по анализу работодателей
Компании с наибольшим количеством вакансий сконцентрированы в крупных экономических центрах. IT-сфера занимает лидирующие позиции, особенно разработка ПО. Некоторые регионы имеют много работодателей, но без вакансий, что говорит о дисбалансе рынка труда.

# Юнит 6. Предметный анализ

1. Сколько вакансий имеет отношение к данным?

Считаем, что вакансия имеет отношение к данным, если в её названии содержатся слова 'data' или 'данн'.

*Подсказка: Обратите внимание, что названия вакансий могут быть написаны в любом регистре.* 


In [111]:
# текст запроса
query_6_1 = '''
SELECT count(*)
FROM vacancies
WHERE LOWER(name) LIKE '%data%' OR LOWER(name) LIKE '%данн%';
'''

In [112]:
# результат запроса
connection = engine.raw_connection()
df = pd.read_sql_query(query_6_1, connection)
connection.close()
print('Вакансии имеющие отношение к данным:', df['count'].values[0])

/var/folders/6k/1096gq_54_dgk0_cjz4762g40000gn/T/ipykernel_54159/2947098479.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query_6_1, connection)


Вакансии имеющие отношение к данным: 1771


2. Сколько есть подходящих вакансий для начинающего дата-сайентиста? 
Будем считать вакансиями для дата-сайентистов такие, в названии которых есть хотя бы одно из следующих сочетаний:
* 'data scientist'
* 'data science'
* 'исследователь данных'
* 'ML' (здесь не нужно брать вакансии по HTML)
* 'machine learning'
* 'машинн%обучен%'

** В следующих заданиях мы продолжим работать с вакансиями по этому условию.*

Считаем вакансиями для специалистов уровня Junior следующие:
* в названии есть слово 'junior' *или*
* требуемый опыт — Нет опыта *или*
* тип трудоустройства — Стажировка.
 

In [125]:
# текст запроса
ds_filter = '''
(name ILIKE '%data scientist%'
    OR name ILIKE '%data science%'
    OR name ILIKE '%исследователь данных%'
    OR name LIKE '%ML%' AND name NOT ILIKE '%HTML%'
    OR name ILIKE '%machine learning%'
    OR name ILIKE '%машинн%обучен%')
'''
query_6_2 = f'''
select count(*)
from vacancies
where {ds_filter}
  and (
    name ilike '%junior%'
        or experience = 'Нет опыта'
        or employment = 'Стажировка'
    );
'''

In [126]:
# результат запроса
connection = engine.raw_connection()
df = pd.read_sql_query(query_6_2, connection)
connection.close()
print('Подходящие вакансии для начинающих дата-сайентистов:', df['count'].values[0])

/var/folders/6k/1096gq_54_dgk0_cjz4762g40000gn/T/ipykernel_54159/2296427000.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query_6_2, connection)


Подходящие вакансии для начинающих дата-сайентистов: 51


3. Сколько есть вакансий для DS, в которых в качестве ключевого навыка указан SQL или postgres?

** Критерии для отнесения вакансии к DS указаны в предыдущем задании.*

In [133]:
# текст запроса
query_6_3 = f'''
select count(*)
from vacancies
where (name ILIKE '%data scientist%'
    OR name ILIKE '%data science%'
    OR name ILIKE '%исследователь данных%'
    OR name ILIKE '%ML%' AND name NOT ILIKE '%HTML%'
    OR name ILIKE '%machine learning%'
    OR name ILIKE '%машинн%обучен%')
  and (key_skills ilike '%SQL%' or key_skills ilike '%postgres%');
'''

In [134]:
# результат запроса
connection = engine.raw_connection()
df = pd.read_sql_query(query_6_3, connection)
connection.close()
print('Вакансии для DS, в которых в качестве ключевого навыка указан SQL или postgres:', df['count'].values[0])

/var/folders/6k/1096gq_54_dgk0_cjz4762g40000gn/T/ipykernel_54159/346592134.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query_6_3, connection)


Вакансии для DS, в которых в качестве ключевого навыка указан SQL или postgres: 229


4. Проверьте, насколько популярен Python в требованиях работодателей к DS.Для этого вычислите количество вакансий, в которых в качестве ключевого навыка указан Python.

** Это можно сделать помощью запроса, аналогичного предыдущему.*

In [135]:
# текст запроса
query_6_4 = f'''
select count(*)
from vacancies
where (name ILIKE '%data scientist%'
    OR name ILIKE '%data science%'
    OR name ILIKE '%исследователь данных%'
    OR name ILIKE '%ML%' AND name NOT ILIKE '%HTML%'
    OR name ILIKE '%machine learning%'
    OR name ILIKE '%машинн%обучен%')
  and key_skills ilike '%Python%';
'''

In [136]:
# результат запроса
connection = engine.raw_connection()
df = pd.read_sql_query(query_6_4, connection)
connection.close()
print('Количество вакансий, в которых в качестве ключевого навыка указан Python:', df['count'].values[0])

/var/folders/6k/1096gq_54_dgk0_cjz4762g40000gn/T/ipykernel_54159/4276501372.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query_6_4, connection)


Количество вакансий, в которых в качестве ключевого навыка указан Python: 357


5. Сколько ключевых навыков в среднем указывают в вакансиях для DS?
Ответ округлите до двух знаков после точки-разделителя.

In [131]:
# текст запроса
query_6_5 = f'''
select
round(avg(length(key_skills) - length(replace(key_skills,CHR(9),''))+1),2)

from vacancies v

WHERE
    {ds_filter}
    AND (key_skills IS NOT NULL)
'''

In [132]:
# результат запроса
connection = engine.raw_connection()
df = pd.read_sql_query(query_6_5, connection)
connection.close()
print('Количество ключевых навыков в среднем указанных в вакансиях для DS:', df['round'].values[0])

/var/folders/6k/1096gq_54_dgk0_cjz4762g40000gn/T/ipykernel_54159/2078762431.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query_6_5, connection)


Количество ключевых навыков в среднем указанных в вакансиях для DS: 6.41


6. Напишите запрос, позволяющий вычислить, какую зарплату для DS в **среднем** указывают для каждого типа требуемого опыта (уникальное значение из поля *experience*). 

При решении задачи примите во внимание следующее:
1. Рассматриваем только вакансии, у которых заполнено хотя бы одно из двух полей с зарплатой.
2. Если заполнены оба поля с зарплатой, то считаем зарплату по каждой вакансии как сумму двух полей, делённую на 2. Если заполнено только одно из полей, то его и считаем зарплатой по вакансии.
3. Если в расчётах участвует null, в результате он тоже даст null (посмотрите, что возвращает запрос select 1 + null). Чтобы избежать этой ситуацию, мы воспользуемся функцией [coalesce](https://postgrespro.ru/docs/postgresql/9.5/functions-conditional#functions-coalesce-nvl-ifnull), которая заменит null на значение, которое мы передадим. Например, посмотрите, что возвращает запрос `select 1 + coalesce(null, 0)`

Выясните, на какую зарплату в среднем может рассчитывать дата-сайентист с опытом работы от 3 до 6 лет. Результат округлите до целого числа. 

In [139]:
# текст запроса
query_6_6 = f'''
select experience, round(avg((coalesce(salary_to, salary_from) + coalesce(salary_from, salary_to)) / 2))
from vacancies
where {ds_filter}
  and (salary_to is not null
    or salary_from is not null)
group by experience;
'''

In [142]:
# результат запроса
connection = engine.raw_connection()
df = pd.read_sql_query(query_6_6, connection)
connection.close()
print('Зарплата в среднем на которую может рассчитывать дата-сайентист с опытом работы от 3 до 6 лет:', df['round'].values[2])

/var/folders/6k/1096gq_54_dgk0_cjz4762g40000gn/T/ipykernel_54159/1675807443.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query_6_6, connection)


Зарплата в среднем на которую может рассчитывать дата-сайентист с опытом работы от 3 до 6 лет: 243115.0


***

### Выводы по предметному анализу
- Вакансий, связанных с данными, много — подтверждается высокий спрос на специалистов в области Data Science.
- Для начинающих специалистов меньше вакансий, что указывает на упор рынка на опытных кандидатов.
- Наиболее востребованные навыки: **Python** и **SQL/Postgres**.
- В среднем в вакансиях указывается около 6 ключевых навыков.
- Дата-сайентисты с опытом 3-6 лет получают высокую среднюю зарплату, что показывает ценность специалистов этого уровня.

### Общий вывод по проекту
В ходе исследования базы вакансий HeadHunter было выявлено следующее:
1. **Данные о вакансиях:**
    - В базе представлено большое количество вакансий, распределенных по регионам и сферам деятельности, что отражает разнообразие рынка труда.
    - Вакансии сосредоточены преимущественно в крупных экономических центрах.

2. **Анализ работодателей:**
    - Лидирующую роль на рынке занимают крупные IT-компании.
    - Разработка программного обеспечения — одна из наиболее востребованных сфер деятельности.
    - Некоторые регионы демонстрируют дисбаланс: много работодателей без актуальных вакансий.

3. **Предметный анализ Data Science:**
    - На рынке труда высокая востребованность специалистов, связанных с обработкой данных и Data Science.
    - Востребованные навыки: **Python**, **SQL/Postgres**, знания в области машинного обучения.
    - Для начинающих специалистов рынок предлагает ограниченное количество вакансий, делая акцент на опыте.
    - Средний уровень зарплат для DS-специалистов с опытом работы от 3 до 6 лет подтверждает ценность таких специалистов на рынке.


### **Региональное сравнение**
- **Цель:** Понять, как регион влияет на количество вакансий, зарплаты и востребованные навыки.
- **Исследования:**
    - Уровень доступности вакансий для начинающих специалистов в разных регионах.
- **Прогноз:** Выявить перспективные регионы для развития карьер junior специалистов.


In [147]:
query_7 = '''SELECT
    a.name AS region,
    COUNT(v.id) AS junior_vacancies_count,
    ROUND((COUNT(v.id)::DECIMAL / total_vacancies) * 100, 2) AS junior_vacancy_percentage
FROM areas a
         JOIN vacancies v ON a.id = v.area_id
         LEFT JOIN (
    SELECT area_id, COUNT(id) AS total_vacancies
    FROM vacancies
    GROUP BY area_id
) tv ON a.id = tv.area_id
WHERE (LOWER(v.name) LIKE '%junior%' OR v.experience = 'Нет опыта' OR LOWER(v.employment) LIKE '%стажировка%')
GROUP BY a.name, total_vacancies
ORDER BY junior_vacancy_percentage DESC;
'''

In [149]:
connection = engine.raw_connection()
df = pd.read_sql_query(query_7, connection)
connection.close()
print(df)

/var/folders/6k/1096gq_54_dgk0_cjz4762g40000gn/T/ipykernel_54159/4060149311.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query_7_1, connection)


         region  junior_vacancies_count  junior_vacancy_percentage
0    Рассказово                       1                     100.00
1         Кашин                       3                     100.00
2     Кировград                       1                     100.00
3      Осташков                       1                     100.00
4      Хотьково                       1                     100.00
..          ...                     ...                        ...
458      Польша                       2                       3.77
459      Батуми                       3                       3.61
460     Армения                       8                       3.59
461        Баку                       3                       3.30
462   Иннополис                       1                       1.10

[463 rows x 3 columns]


**Результат анализа:** Этот запрос позволяет увидеть, где начинающим специалистам проще найти подходящие вакансии, и оценить регионы с лучшими условиями для старта карьеры.
